##### Author contributions
Please fill out for each of the following parts who contributed to what:
- Conceived ideas: 
- Performed math exercises: 
- Performed programming exercises:
- Contributed to the overall final assignment: 

# Chapter 7
## Recurrent neural networks


    Hand-in bug-free (try "Kernel" > "Restart & Run All") and including all (textual as well as figural) output via Brightspace before the deadline (see Brightspace).

Learning goals:
1. Get familiar with recurrent hidden units
1. Implement a simple RNN (Elman network) in PyTorch
1. Implement an LSTM-based neural network in PyTorch

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

### Exercise 1  (1 point)

Consider a recurrent neural network with one input unit $x$, one sigmoid recurrent hidden unit $h$, and one linear output unit $y$. The values of $x$ are given for 3 time points in `x_t`. As this is a very small RNN, $W^i$, $W^h$ and $W^o$ are given as the scalar values `w_i`, `w_h` and `w_o` respectively. The hidden unit has an added bias `h_bias`. The hidden unit state is initialized with `0.0`. The only 'value-manipulating' activation function in this network is the sigmoid activation $\sigma(\cdot)$ on the hidden unit. 

1. Write down the forward pass of this network for a specific time point $t$. 
1. What is the value of the hidden state $h$ after processing the last input `x_t[2]`? 
1. What is the output `y` of the network after processing the last input `x_t[2]`? 

\begin{eqnarray*}
h_t &=& \\ 
y_t &=&  \\
\end{eqnarray*}


For 1.2 and 1.3, you can either compute the solution by hand (show clearly how you arrived there, 3 decimal points) or write code to find the answer. 

In [ ]:
# inputs over times 0, 1, 2:
x_t = [9.0, 4.0, -2.0]

# weights and bias terms: 
w_i = 0.5
w_h = -1.0
w_o = -0.7
h_bias = -1.0
y_bias = 0.0

### Solution 1

In [ ]:
## LaTex or code here ##

### Code introduction

We will apply two recurrent neural networks to learn a dynamic variant of the *adding problem*. First, run the next cell and inspect the output. 

There is a stream of inputs to the network, two at each time step. The first input unit will receive a series of decimal numbers in the interval $[-1,1]$. The second input unit will receive the numbers $0$, $-1$, or $1$. The target is the sum of the preceding two decimal numbers that came together with the number $1$ (called the marker, `x` in the generated output), and it should be produced whenever a marker has been seen. In the beginning until two of these markers have been seen, the output will stay 0. 


Below you will find two functions: 
1. `create_addition_data`: Generates sequential training data sets `X` and `T` for the dynamic *adding problem*, returns numpy array.
1. `MyDataset`: a custom PyTorch dataset that makes sure dimensions are as PyTorch likes them, and can return individual samples the way PyTorch wants them.

Note, the data are represented in a dictionary called `data`. To access the training, validation, and testing data, you can call `data["train"]`, `data["valid]`, and `data["test"]` respectively.

In [ ]:
def create_addition_data(n_samples=3000):
    # This is a dynamic variant of the adding problem. 
    
    # Random numbers in [-1.0,1.0]): 
    X1 = np.random.uniform(low=-1.0, high=1.0, size=(n_samples,) )   
    
    # Random markers [-1.0, 0.0, 1.0] (1.0 marks the numbers that should be added):
    X2 = np.random.choice([-1.0, 0.0, 1.0], size=(n_samples,), p=[0.25, 0.25, 0.5])
    
    # Combine
    X = np.vstack((X1, X2)).T.astype("float32")

    # Create targets
    T = np.zeros((n_samples, 1)).astype("float32")

    # Get indices of 1.0
    markers = np.nonzero(X2 == 1.0)[0]
    
    # Generate data
    mem = X1[markers[0]]
    for mi, marker in enumerate(markers[1:]):
        T[marker] = mem + X1[marker]
        mem = X1[marker]
                
    return X, T

In [ ]:
# Long as the markers x are sparse
X, T = create_addition_data(n_samples=100)

# Print some data
print("Data for the adding problem (x marks 1.0):")
for t in range(X.shape[0]):
    print("Time: {:03d} \t x: ({:+.3f} , {}) \t t: {:+.3f} ".format(
        t, X[t,0], 'x' if X[t,1] == 1.0 else ' ', T[t,0]))

In [ ]:
# Make PyTorch dataset
class MyDataset(torch.utils.data.Dataset):
    
    def __init__(self, X, T):
        self.X = torch.from_numpy(X).type(torch.FloatTensor) # [n_examples, n_samples, n_features]
        self.T = torch.from_numpy(T).type(torch.FloatTensor) # [n_examples, n_samples]
        
    def __getitem__(self, index):
        return self.X[index, :, :], self.T[index]
    
    def __len__(self):
        return self.X.size()[0]

In [ ]:
n_examples = 9
n_samples = 3000
data = {}

# Define training data
X = np.zeros((n_examples, n_samples, 2))
T = np.zeros((n_examples, n_samples, 1))
for i_example in range(n_examples):
    X[i_example, :, :], T[i_example, :] = create_addition_data(n_samples)
data["train"] = torch.utils.data.DataLoader(MyDataset(X, T), batch_size=3)

# Define validation data
X = np.zeros((n_examples, n_samples, 2))
T = np.zeros((n_examples, n_samples, 1))
for i_example in range(n_examples):
    X[i_example, :, :], T[i_example, :] = create_addition_data(n_samples)
data["valid"] = torch.utils.data.DataLoader(MyDataset(X, T), batch_size=3)

# Define test data
X = np.zeros((n_examples, n_samples, 2))
T = np.zeros((n_examples, n_samples, 1))
for i_example in range(n_examples):
    X[i_example, :, :], T[i_example, :] = create_addition_data(n_samples)
data["test"] = torch.utils.data.DataLoader(MyDataset(X, T), batch_size=1)

### Exercise 2: training a network  (0.5 points)

We neede a function to train a `model`. This function `train_model(model, data, optimizer, criterion, n_epochs)` should do the following: 

1. Loop `n_epochs` times over the dataset, and loop over minibatches
1. Train the model on the training data and save the loss per epoch
1. Validate the model on the validation data and save the loss per epoch
1. The function should return the trained model and the losses

Note: this function is quite similar again as the function your wrote wor the MLP and CNN. The only difference is that we do not need to compute an accuracy, as we are performing regression here.

### Solution 2

In [ ]:
## Code here ##

### Exercise 3: Testing a network  (1.5 points)

We neede a function to test a trained `model`. This function `test_model(model, data)` should do the following: 

1. Let `model` predict outputs on the testing data. For this, iterate through test data `data["test]` and pass each sample through `model`. 
1. Save the model output as well as the target output
1. The function should return the predicted and target outputs

### Solution 3

In [ ]:
## Code here ##

### Exercise 4: Simple RNN  (3 points)

We first implement a simple recurrent architecture (a simple [Elman network](http://mnemstudio.org/neural-networks-elman.htm)). 

1. First implement the linear layers `l1` and `l2`. They should lead from `n_input` input units over `n_hidden` hidden units to `n_out` output units.
1. Add a recurrent linear weight layer `hr`. These are weights that self-connect to the hidden units. The input will be the values of the `n_hidden` hidden units, and they should project back to the `n_hidden` hidden units. 
1. A forward pass will update the hidden state with the inputs and the recurrent layer weights, and produce the output from the hidden unit. Specifically you should do the following: 
    2. If we are at the first time point, the hidden state should be set to the input passed through `l1` and `tanh` activations.
    2. If the hidden state has information from previous time points: a) Pass the input through `l1`. b) Pass the hidden state through the recurrent weight layer `hr`. c) The sum of a) and b) should be passed through the `tanh` activation. d) The result should be the new hidden state (used for the next time point).
    2. Finally pass the hidden state through layer `l2`. This produces the output `y` for that time point.
1. The forward pass will receive data `x` with shape [batch_size, time_points, features]. So within the forward pass, you will have to loop over time points, performing the steps as descibed above. The output of the forward pass is then output `y` with shape [batch_size, time_points, 1].

Note: this exercise could also be done with nn.RNN(). However, we want you to understand what a RNN is doing, so we want you to use nn.Linear instead.

### Solution 4

In [ ]:
## Code here ##

### Exercise 5: Setup and run (1 point)

Try your simple `RNN` with the dynamic addition task. 

1. Define the model. `RNN` should have **2 hidden units**.
1. Define the loss as the Mean Squared Error loss, and use an Adam optimizer.
1. Train your model for several epochs on the data with `train_model`.
1. Plot the train and validation losses. 
1. Test the trained model with `test_model` 
1. Plot at least one target time series together with the predicted time series

Based on the losses and predictions, what would your conclusion be? Did the simple RNN learn the task? 

### Solution 5

In [ ]:
## Code here ##

### Exercise 6: LSTM RNN (2 points)

Long-Short Term Memory (LSTM) units have more [powerful functionality](http://colah.github.io/posts/2015-08-Understanding-LSTMs/), such as selective forgetting, and they are able to keep track of long-term dependencies. This might be useful for the adding task. 

Implement the `LSTM` model:

1. `lstm` should be an `LSTM` layer leading from the `n_input` inputs to the `n_hidden` hidden units.
1. `fc` should be a fully-connected (linear) layer leading from the hidden units (output of `lstm`) to the `n_out` output units. 
1. The network does not make use of further activation functions. 

### Solution 6

In [ ]:
## Code here ##

### Exercise 7: Setup and run (1 point)

Try your `LSTM` model with the dynamic addition task. 

1. Define the model. `LSTM` should have **2 hidden units**.
1. Define the loss as the Mean Squared Error loss, and use an Adam optimizer.
1. Train your model for several epochs on the data with `train_model`.
1. Plot the train and validation losses. 
1. Test the trained model with `test_model` 
1. Plot at least one target time series together with the predicted time series

Did the LSTM network capture the task better? Did any of the two capture the task perfectly? Or are the two networks on par? 

### Solution 7

In [ ]:
## Code here ##